# Outage Duration Prediction

**Name(s)**: Neil Sharma, Xiang Ding

**Website Link**: (your website link)

## Code

In [4]:
#Default Libraries
import os

#Third Party Libraries
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

#Sklearn
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import (HuberRegressor, LinearRegression,
                                  QuantileRegressor, SGDRegressor)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (FunctionTransformer, OneHotEncoder,
                                   QuantileTransformer, StandardScaler)

pd.options.plotting.backend = 'plotly'

### Framing the Problem

Prediction Problem Type: This problem type is regression, as we are trying to predict a continuous quantity, in this case how long an outage occurs.

Response Variable: Our response variable will be, in minutes, how long an outage occurs for. This value can be seen in the dataset as 'OUTAGE.DURATION'

Metric: We will use mean squared error (MSE) loss as our metric for evaluation. We decided on MSE due to its sensitivity to outliers and its ability to be easily understood as it maintains the same units as our response variable, 'OUTAGE.DURATION'.

In [89]:
##################################################
#     DATA CLEANING CODE FROM PROJECT 3          #
##################################################
df = pd.read_excel('outage.xlsx', skiprows = 5)

df = df.set_index('OBS')
df = df.iloc[1: , :]
df = df[df.columns[1:]]

#Convert to pd.to_datetime
df['OUTAGE.START.DATE'] = pd.to_datetime(df['OUTAGE.START.DATE'])
df['OUTAGE.START.DATE'] = df['OUTAGE.START.DATE'].dt.date
df['OUTAGE.RESTORATION.DATE'] = pd.to_datetime(df['OUTAGE.RESTORATION.DATE'])
df['OUTAGE.RESTORATION.DATE'] = df['OUTAGE.RESTORATION.DATE'].dt.date
df['CUSTOMERS.AFFECTED_MISSING'] = df['CUSTOMERS.AFFECTED'].isna().astype(int)


#Grabbing columns we think are needed and dropping rows with nan values for response variable
df = df[['MONTH', 'OUTAGE.START.TIME', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY', 'U.S._STATE', 
            'NERC.REGION', 'CLIMATE.REGION', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL', 'POPULATION',
            'POPPCT_URBAN', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL', 'AREAPCT_URBAN', 'AREAPCT_UC',
            'RES.PRICE', 'COM.PRICE', 'IND.PRICE', 'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES',
            'PI.UTIL.OFUSA', 'HURRICANE.NAMES', 'OUTAGE.DURATION', 'CUSTOMERS.AFFECTED']]
df = df.dropna(subset=['OUTAGE.DURATION'])

#Categorize the times for morning and afternoon/evening
def categorize_time(time_str):
    if pd.isna(time_str):
        return 'Unknown'  
    time = pd.to_datetime(time_str, format='%I:%M:%S %p', errors='coerce')
    if time.hour < 12:
        return 'Morning'
    else:
        return 'Afternoon/Evening'

# Apply the function to create new columns
df['OUTAGE.START.CATEGORY'] = df['OUTAGE.START.TIME'].apply(categorize_time)
#df['OUTAGE.END.CATEGORY'] = df['OUTAGE.RESTORATION.TIME'].apply(categorize_time)


pd.set_option('display.max_columns', None)
df

,MONTH,OUTAGE.START.TIME,ANOMALY.LEVEL,CLIMATE.CATEGORY,U.S._STATE,NERC.REGION,CLIMATE.REGION,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,POPULATION,POPPCT_URBAN,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,PI.UTIL.OFUSA,HURRICANE.NAMES,OUTAGE.DURATION,CUSTOMERS.AFFECTED,OUTAGE.START.CATEGORY
OBS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,7.0,17:00:00,-0.3,normal,Minnesota,MRO,East North Central,severe weather,NaN,5348119.0,73.27,2279,1700.5,18.2,2.14,0.6,11.6,9.18,6.81,9.28,2332915,2114774,2113291,6562520,2.2,NaN,3060,70000.0,Afternoon/Evening
2.0,5.0,18:38:00,-0.1,normal,Minnesota,MRO,East North Central,intentional attack,vandalism,5457125.0,73.27,2279,1700.5,18.2,2.14,0.6,12.12,9.71,6.49,9.28,1586986,1807756,1887927,5284231,2.2,NaN,1,NaN,Afternoon/Evening
3.0,10.0,20:00:00,-1.5,cold,Minnesota,MRO,East North Central,severe weather,heavy wind,5310903.0,73.27,2279,1700.5,18.2,2.14,0.6,10.87,8.19,6.07,8.15,1467293,1801683,1951295,5222116,2.1,NaN,3000,70000.0,Afternoon/Evening
4.0,6.0,04:30:00,-0.1,normal,Minnesota,MRO,East North Central,severe weather,thunderstorm,5380443.0,73.27,2279,1700.5,18.2,2.14,0.6,11.79,9.25,6.71,9.19,1851519,1941174,1993026,5787064,2.2,NaN,2550,68200.0,Afternoon/Evening
5.0,7.0,02:00:00,1.2,warm,Minnesota,MRO,East North Central,severe weather,NaN,5489594.0,73.27,2279,1700.5,18.2,2.14,0.6,13.07,10.16,7.74,10.43,2028875,2161612,1777937,5970339,2.2,NaN,1740,250000.0,Afternoon/Evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1527.0,3.0,00:00:00,1.6,warm,Idaho,WECC,Northwest,intentional attack,sabotage,1680026.0,70.58,2216.8,2004.7,5.6,0.6,0.19,9.8,7.66,5.98,8.05,678472,499372,479761,1657605,0.4,NaN,0,0.0,Afternoon/Evening
1529.0,7.0,15:45:00,-0.3,normal,Idaho,WECC,Northwest,system operability disruption,uncontrolled loss,1680026.0,70.58,2216.8,2004.7,5.6,0.6,0.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4,NaN,220,NaN,Afternoon/Evening
1530.0,12.0,08:00:00,-0.9,cold,North Dakota,MRO,West North Central,public appeal,NaN,685326.0,59.9,2192.2,1868.2,3.9,0.27,0.1,8.41,7.8,6.2,7.56,488853,438133,386693,1313678,0.5,NaN,720,34500.0,Afternoon/Evening


### Baseline Model

In [100]:
#############################################
#             BASELINE MODEL                #
#    Using two features,Linear Regression   #
#               and MAE loss                #
#############################################

#Select our two features and response variable
X = df[['ANOMALY.LEVEL','CAUSE.CATEGORY']]
y = df['OUTAGE.DURATION']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

# Preprocessing for numerical data: no transformation needed
# Preprocessing for categorical data: OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['CAUSE.CATEGORY']),
        ('standard', StandardScaler(), ['ANOMALY.LEVEL'])
    ])

# Create a pipeline
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

# Train the model
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 2666.5076863993104


In [27]:
#################################
#     PLOT BASELINE MODEL       #
#################################

trace0 = go.Scatter(
    x=np.arange(len(y_test)),
    y=y_test,
    mode='markers',
    name='Actual Values'
)
trace1 = go.Scatter(
    x=np.arange(len(y_pred)),
    y=y_pred,
    mode='markers',
    name='Predicted Values'
)

# Create layout
layout = go.Layout(
    title='Actual vs Predicted Values',
    xaxis=dict(title='Index'),
    yaxis=dict(title='Outage Duration')
)

# Create figure and add traces
fig = go.Figure(data=[trace0, trace1], layout=layout)

# Show plot
fig.show()

### Final Model

In [98]:
########################################
#              FINAL MODEL             #
# We have chosen to employ 10 features #
# for our model, 4 of which are numeric#
# and the other 6 of which are         #
# categorical.                         #
########################################

features = ['NERC.REGION', 'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY', 'CAUSE.CATEGORY',
            'CUSTOMERS.AFFECTED', 'POPULATION', 'U.S._STATE', 'CAUSE.CATEGORY.DETAIL',
            'PI.UTIL.OFUSA']
target = 'OUTAGE.DURATION'

X = df[features]
y = df[target]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Preprocessing
numeric_features = ['ANOMALY.LEVEL', 'PI.UTIL.OFUSA', 'CUSTOMERS.AFFECTED', 'POPULATION']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('quantile', QuantileTransformer(n_quantiles=min(675, len(X_train)), output_distribution='uniform'))
])

categorical_features = ['NERC.REGION', 'CLIMATE.REGION', 'CLIMATE.CATEGORY', 'CAUSE.CATEGORY', 'U.S._STATE', 'CAUSE.CATEGORY.DETAIL']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# We are choosing Random Forest Regressor for its ability to determine complex patterns and complexity
model = RandomForestRegressor()

# Pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Hyperparameter Tuning for RandomForestRegressor
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [None, 10, 20, 30],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4]
    # Add other RandomForestRegressor specific parameters if needed
}
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error')

# Fit the model
grid_search.fit(X_train, y_train)

# Predict using the model
y_pred = grid_search.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
print("Best parameters:", grid_search.best_params_)
print("Best score (CV):", -grid_search.best_score_)
print("Test Mean Absolute Error:", mae)

Best parameters: {'model__max_depth': 20, 'model__min_samples_leaf': 2, 'model__min_samples_split': 10, 'model__n_estimators': 100}
Best score (CV): 2118.2467898850437
Test Mean Absolute Error: 1986.9464234368163


In [99]:
#################################
#     PLOT FINAL MODEL          #
#################################

trace0 = go.Scatter(
    x=np.arange(len(y_test)),
    y=y_test,
    mode='markers',
    name='Actual Values'
)
trace1 = go.Scatter(
    x=np.arange(len(y_pred)),
    y=y_pred,
    mode='markers',
    name='Predicted Values'
)

# Create layout
layout = go.Layout(
    title='Actual vs Predicted Values',
    xaxis=dict(title='Index'),
    yaxis=dict(title='Outage Duration')
)

# Create figure and add traces
fig = go.Figure(data=[trace0, trace1], layout=layout)

# Show plot
fig.show()

### Fairness Analysis

Does our model work better on normal vs. warm and cold CLIMATE.CATEGORY

In [104]:
##############################
#       FAIRNESS ANALYSIS    #
##############################

features = ['NERC.REGION', 'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY', 'CAUSE.CATEGORY',
            'CUSTOMERS.AFFECTED', 'POPULATION', 'U.S._STATE', 'CAUSE.CATEGORY.DETAIL',
            'PI.UTIL.OFUSA']
target = 'OUTAGE.DURATION'

X = df[features]
y = df[target]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Split the test data based on climate category
normal_data = X_test[X_test['CLIMATE.CATEGORY'] == 'normal']
warm_cold_data = X_test[X_test['CLIMATE.CATEGORY'].isin(['warm', 'cold'])]

# Predict and calculate metrics for each group
y_pred_normal = grid_search.predict(normal_data)
y_pred_warm_cold = grid_search.predict(warm_cold_data)

rmse_normal = np.sqrt(mean_squared_error(y_test.loc[normal_data.index], y_pred_normal))
rmse_warm_cold = np.sqrt(mean_squared_error(y_test.loc[warm_cold_data.index], y_pred_warm_cold))

# Permutation test
n_permutations = 1000
rmse_diffs = []

for _ in range(n_permutations):
    # Shuffle the CLIMATE.CATEGORY values
    shuffled_category = np.random.permutation(X_test['CLIMATE.CATEGORY'])
    
    # Split the data based on shuffled categories
    normal_shuffled = X_test[shuffled_category == 'normal']
    warm_cold_shuffled = X_test[np.isin(shuffled_category, ['warm', 'cold'])]
    
    # Predict and calculate metrics
    y_pred_normal_shuffled = grid_search.predict(normal_shuffled)
    y_pred_warm_cold_shuffled = grid_search.predict(warm_cold_shuffled)
    
    rmse_normal_shuffled = np.sqrt(mean_squared_error(y_test.loc[normal_shuffled.index], y_pred_normal_shuffled))
    rmse_warm_cold_shuffled = np.sqrt(mean_squared_error(y_test.loc[warm_cold_shuffled.index], y_pred_warm_cold_shuffled))

    # Compute the difference in RMSE
    diff = rmse_normal_shuffled - rmse_warm_cold_shuffled
    rmse_diffs.append(diff)

# Analyze the permutation test results
observed_diff = rmse_normal - rmse_warm_cold
p_value = np.sum(np.abs(rmse_diffs) >= np.abs(observed_diff)) / n_permutations

# Results
print(f"Observed difference in RMSE: {observed_diff}")
print(f"P-value: {p_value}")

Observed difference in RMSE: 410.7924950725451
P-value: 0.437


In [108]:
#################################
#   PLOT FAIRNESS ANALYSIS      #
#################################

# Create a histogram of the permutation differences
fig = go.Figure(data=[go.Histogram(x=rmse_diffs, nbinsx=40, name='Permutation Differences')])

# Mark the observed difference
fig.add_trace(go.Scatter(
    x=[observed_diff],
    y=[0],
    mode='markers',
    marker=dict(color='red', size=10),
    name='Observed Difference'
))

# Update layout for readability
fig.update_layout(
    title='Permutation Test for RMSE Differences',
    xaxis_title='RMSE Difference',
    yaxis_title='Frequency',
    bargap=0.2
)

# Show the plot
fig.show()
